In [ ]:
from dotenv import dotenv_values
import os
import pandas
import requests
import json
import types
import time 


def get_places(api_key: str, url: str, query: str, page_token:str = None):
    """
    Get places from Google Places API.

    Args:
        api_key: str - Google Places API key
        url: str - Google Places API URL
        query: str - name of the city
    """

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "nextPageToken,places.id,places.name,places.displayName,places.formattedAddress,places.priceLevel,places.attributions,places.location,places.priceLevel,places.rating,places.userRatingCount,places.servesBreakfast,places.servesBrunch,places.servesCoffee,places.servesDessert,places.servesDinner,places.servesLunch,places.servesVegetarianFood,places.goodForChildren,places.allowsDogs,places.googleMapsLinks",
    }

    data = {"textQuery": query}

    if page_token:
        data["pageToken"] = page_token
  
    try:
        respone = requests.post(url, headers=headers, json=data, timeout=10)
        respone.raise_for_status()
        return respone.json()

    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
        return None

    except Exception as e:
        print(f"Unexpected error occurred: {e}")
        return None





In [ ]:



data = get_places(API_KEY, URL_GOOGLE_APIS_PLACE, query="restaurants in Poznań")


In [65]:
def get_all_city_places(api_key: str, url: str, query: str, max_results: int, page_token: str = None) -> list: 
    """ 
    Get all places for a given query.
    Args:
        api_key: str - Google Places API key
        url: str - Google Places API URL
        query: str - name of the city
        max_results: int - maximum number of results    
    """
    results = []

    while len(results)< max_results:
        print(page_token)
        data = get_places(api_key, url, query, page_token)
        if 'places' in data:
            results.extend(data['places'])
        else:
            break
        
        if data.get('nextPageToken', None):
            page_token = data['nextPageToken']
        else: 
            break
        time.sleep(2)

    return results



In [ ]:
data = get_all_city_places(API_KEY, URL_GOOGLE_APIS_PLACE, query="restaurants in Poznań", max_results=60)

60

In [68]:
def get_all(api_key: str, url: str, query: list,  max_results: int = 40) -> list:
    """
    Get places from all the cities in the list. 
    Args:
        api_key: str - Google Places API key
        url: str - Google Places API URL 
        query: list - list of cities
    
    """
    results = []
    for city in query:
        data = get_all_city_places(api_key, url, city, max_results)
        results.extend(data)
    
    return results
        



In [ ]:
city_restaurants_list = ['restaurants in Poznań', 'restaurants in Kraków']
all_cities_restaurants = get_all(API_KEY, URL_GOOGLE_APIS_PLACE, query=city_restaurants_list, max_results=40)

80